# Loterias

## Análises

### Preparo

#### Imports

In [ ]:
from time import time
t0: float = time()
from requests import get, Response, HTTPError
t1: float = time()
from io import BytesIO # pula escrita para .xlsx
t2: float = time()
from pandas import read_excel, DataFrame, NA, Series
t3: float = time()
from urllib.parse import quote
t4: float = time()
import matplotlib.pyplot as plt
t5: float = time()

print(
    f"requests: {t1-t0:.3f}s", f"io: {t2-t1:.3f}s", f"pandas: {t3-t2:.3f}s",
    f"urllib: {t4-t3:.3f}s", f"plt: {t5-t4:.3f}s",
    sep='\n'
)

#### Constantes

In [ ]:
modalidade: str

API_ROUTE: str = ("https://servicebus2.caixa.gov.br/portaldeloterias"
                  "/api/resultados/download?modalidade=")
MODALIDADES: list[str] = [
    "Timemania", "Mega-Sena", "Lotofácil", "Lotomania",
    "Quina", "Dupla-Sena", "Dia de Sorte", "Super Sete", "+Milionária"
]
URLS: dict[str, str] = {
    modalidade: API_ROUTE + quote(modalidade)
    for modalidade in MODALIDADES
}
URLS["Lotomania"]

#### Funções

In [ ]:
def resultados(modalidade: str) -> DataFrame:
    """
    Função que retorna um DataFrame dos resultados de uma loteria

    Args:
        modalidade (str):
            A modalidade da loteria

    Returns:
        DataFrame:
            `DataFrame` caso seja possível pegar a planilha da API da Caixa,
            `None` caso contrário

    Raises:
        KeyError: Caso `modalidade` não seja uma modalidade válida
    """

    if modalidade not in URLS:
        raise KeyError("Modalidade inválida")

    loteria: Response
    resultado: DataFrame

    try:
        loteria = get(URLS[modalidade])
        loteria.raise_for_status()
        resultado = read_excel(BytesIO(loteria.content),
                               index_col="Concurso")
        return resultado
    except HTTPError:
        print("Impossível carregar")
        return DataFrame()
    except ValueError as err:
        print(f"Erro na conversão para DataFrame: {err}")

In [ ]:
def figshow(df: DataFrame, col: str, name: str,
            nrows: int, ncols: int, extra: str = "", yticks: int = 10,
            sizemul: float = 2.5) -> plt.Figure:
    """
    Função que plota os resultados de uma loteria

    Args:
        df (DataFrame):
            O DataFrame com os resultados.

        col (str):
            O nome da coluna principal (e.g.: `"Bola"`).

        name (str):
            O título do plot.

        nrows (int):
            Quantidade de linhas para distribuir os plots.

        ncols (int):
            Quantidade de colunas para distribuir os plots.

        extra (str):
            Nome da coluna de valores extras (e.g.: `"Mês da Sorte"`). Vazio
            por padrão.

        yticks (int):
            Intervalo entre os ticks no eixo vertical. 10 por padrão.

        sizemul (float):
            Multiplica o tamanho original da figura. 2.5 por padrão.

    Returns:
        plt.Figure:
            A figura com os plots dos resultados.
    """

    fig: plt.Figure
    num_cols_bolas: int = len([c for c in df.columns if c.startswith(col)])
    num_cols_extra: int = (len([c for c in df.columns if c.startswith(extra)])
                           if extra else 0)
    fig, _ = plt.subplots(nrows=nrows, ncols=ncols)

    fig.suptitle(name)

    bolas: list[dict[int, int]]
    if extra:
        bolas = ([bola.value_counts().to_dict()
                for bola in (df[c] for c in df.columns
                            if c.startswith((col, extra)))])
    else:
        bolas = ([bola.value_counts().to_dict()
                for bola in (df[c] for c in df.columns
                            if c.startswith(col))])

    max_count: int
    if num_cols_extra:
        max_count = max((max(i.values()) for i in bolas[:-num_cols_extra]))
    else:
        max_count = max((max(i.values()) for i in bolas))

    ranges: tuple[int, int] = (min(min(list(nums.keys())) for nums in bolas),
                               max(max(list(nums.keys())) for nums in bolas) + 1)

    for (ind, ax) in enumerate(fig.axes):
        if ind == (num_cols_bolas + num_cols_extra): break
        ax.bar(x=list(bolas[ind].keys()), height=list(bolas[ind].values()))
        if ind < num_cols_bolas:
            ax.set_xticks(range(*ranges), (str(i) for i in range(*ranges)),
                        rotation=90, fontsize=5)
            ax.set_xlim(ranges[0] - 1, ranges[1])
            ax.set_yticks(range(0, max_count + 1, yticks))
            ax.set_ylim(0, max_count)
            ax.set_title(f"{col.rstrip()} {ind+1}")

        elif ind >= num_cols_bolas:
            t_ranges: tuple[int, int] = (
                min(min(list(nums.keys())) for nums in bolas[-num_cols_extra:]),
                max(max(list(nums.keys())) for nums in bolas[-num_cols_extra:]) + 1)
            t_max_count: int = max((max(i.values()) for i in bolas[-num_cols_extra:]))
            ax.set_xticks(range(*t_ranges), (str(i) for i in range(*t_ranges)))
            ax.set_xlim(t_ranges[0] - 1, t_ranges[1])
            ax.set_yticks(range(0, t_max_count + 1, 10))
            ax.set_ylim(0, t_max_count)
            ax.set_title(f"{extra} {ind - 5}")

    fig.set_size_inches(fig.get_size_inches()*sizemul)

    return fig

### Modalidades

#### Timemania

In [ ]:
# timemania: DataFrame = resultados("Timemania")

# timemania.head()
# timemania.tail()

In [ ]:
# try:
#     cols: list[str] = ([f"Rateio {k} acertos" for k in range(3, 8)]
#             + ["Estimativa Prêmio", "Arrecadação Total",
#                "Acumulado 7 acertos", "Rateio Time Coração"])
#     for c in cols: # conversao para float
#         try:
#             timemania[c] = [
#                 float(i[2:].replace(".", "").replace(",", "."))
#                 for i in timemania[c]
#             ]
#         except ValueError:
#             print(c)
# except TypeError:
#     pass

# try:
#     timemania.drop(columns=["Cidade / UF", "Observação"], inplace=True)
# except KeyError:
#     pass

In [ ]:
# timemania.describe()

In [ ]:
# erros: dict[str, list[str]] = {
#     "ABC/RN": ["A B C/RN"],
#     "ATLÉTICO/MG": ["ATLETICO", "ATLETICOETA"],
#     "ATHLÉTICO/PR": ["ATHLETICOZSEO", "ATHLETICOA"],
#     "ATHLETIC CLUB/MG": ["ATHLETIC CLUB"],
#     "SÃO PAULO/SP": ["SAO PAULO", "S. PAULO/SP", "S PAULO/SP", "S PAULO",
#                      "SAO PAULO", "SAO PAULO/SP", "SAOPAULO"],
#     "TUNA LUSO/PA": ["TUNA LUSO"],
#     "VASCO DA GAMA/RJ": ["VASCO DA GAMA"],
#     "CRB/AL": ["CRB"],
#     "BAHIA/BA": ["BAHIA", "BAHIAICOETA", "BAHIACANOEABA",
#                  "BAHIAMUNDOE", "BAHIARANCO"],
#     "VITÓRIA/BA": ["VITORIA", "VITORIAUJOAOA"],
#     "YPIRANGA/AP": ["YPIRANGA"],
#     "RIVER/PI": ["RIVER", "RIVER PRETAEO", "RIVERNENSEIRA", "RIVERBARENSEA"],
#     "NÁUTICO/PE": ["NAUTICO", "NAUTICODIA"],
#     "SÃO RAIMUNDO/AM": ["SAO RAIMUNDO", "SAORAIMUNDO", "S RAIMUNDO",
#                         "S RAIMUNDOA", "S RAIMUNDOAMA", "S RAIMUNDOE"],
#     "BAHIA DE FEIRA/BA": ["BAHIA DE FEIRA"],
#     "AMÉRICA/RJ": ["AMERICAUJOAOA"],
#     "AMÉRICA/RN": ["AMERICASE"],
#     "AMERICANO/RJ": ["AMERICANO", "AMERICANOEABA"],
#     "PALMEIRAS/SP": ["PALMEIRAS", "PALMEIRASOA", "PALMEIRASESEO"],
#     "CORITIBA/PR": ["CORITIBA", "CORITIBANTINO"],
#     "MIXTO/MT": ["MIXTO", "MIXTORANCO", "MIXTO DA GAMA"],

# }

# # Ambíguos
# ambiguidades: dict[str, dict[str, list[int]]] = {
#     "ATLETICO": {
#         "ATLÉTICO/GO": [1602, 1636, 1653, 1662, 1852, 1905, 1923, 1942,
#                          1948, 1967, 2040, 2065, 2125, 2172, 2242, 2297],
#         "ATLÉTICO/AC": [1908, 1935, 2084],
#         "ATLÉTICO_CE": [2069,],
#         "ATHLÉTICO/PR": [1398, 1526, 1722, 1766, 1808, 1940,
#                           2004, 2045, 2078, 2116, 2165, 2171],
#     },
#     "BOTAFOGO": {
#         "BOTAFOGO/PB": [1546, 1575, 1759, 1840, 1853, 1941, 2051, 2072],
#         "BOTAFOGO/SP": [1789, 1892, 1970, 2031, 2076],
#     },
#     "AMERICA": {
#         "AMÉRICA/RN": [1555, 1690, 1777, 1885, 1897, 2038, 2173, 2205, ],
#         "AMÉRICA/MG": [1571, 1606, 1685, 1831, 1837, 1839, 1953,
#                        2001, 2149, 2150, 2195, 2223, 2245, 2269, ],
#     },

# }

# try:
#     for ambiguidade in ambiguidades:
#         for (time, locs) in ambiguidades[ambiguidade].items():
#             for loc in locs:
#                 timemania.loc[loc, "Time Coração"] = time
# except:
#     pass

# for time in erros:
#     for erro in erros[time]:
#         try:
#             timemania["Time Coração"] = (timemania["Time Coração"]
#                                         .replace(to_replace=erro,
#                                                  value=time))
#         except:
#             continue

# times: dict[str, int] = timemania["Time Coração"].value_counts().to_dict()
# times

#### Mega-Sena

In [ ]:
megasena: DataFrame = resultados("Mega-Sena")

megasena.head()

In [ ]:
try:
	megasena = megasena.drop(columns=["Observação"])
except:
	pass
try:
	for col in megasena.columns:
		if not col.startswith(("Rateio", "Acumulado", "Arrecadação", "Estimativa")):
			continue
		megasena[col] = [
			float(a[2:].replace(".", "").replace(",", ".")) for a in megasena[col]
		]
except:
	pass

In [ ]:
megasena.describe()

In [ ]:
megasena[[f"Bola{i}" for i in range(1, 7)]].describe()

In [ ]:
megasena_fig: plt.Figure = figshow(df=megasena, col="Bola",
                                   name="Mega-Sena", nrows=2, ncols=3)

#### Lotofácil

In [ ]:
lotofacil: DataFrame = resultados("Lotofácil")

lotofacil.head()

In [ ]:
try:
	lotofacil = lotofacil.drop(columns=["Observação"])
except:
	pass
try:
	for col in lotofacil.columns:
		if not col.startswith(("Rateio", "Acumulado", "Arrecadação", "Estimativa")):
			continue
		lotofacil[col] = [
			float(a[2:].replace(".", "").replace(",", ".")) for a in lotofacil[col]
		]
except:
	pass

In [ ]:
lotofacil.describe()

In [ ]:
lotofacil[[f"Bola{i}" for i in range(1, 16)]].describe()

In [ ]:
lotofacil_fig: plt.Figure = figshow(df=lotofacil, col="Bola", name="Lotofácil",
                                    nrows=3, ncols=5, yticks=100)

#### Lotomania

##### Pré-Tratamento

In [ ]:
# Planilha com erro: '.' interpretado como int
# Erro de parsing

##### Análise

In [ ]:
# Planilha com erro: '.' interpretado como int
# Erro de parsing

#### Quina

In [ ]:
quina: DataFrame = resultados("Quina")

quina.head()

In [ ]:
try:
	quina = quina.drop(columns=["observação"])
except:
	pass
try:
	for col in quina.columns:
		if not col.startswith(("Rateio", "Acumulado", "Arrecadação", "Estimativa")):
			continue
		quina[col] = [
			float(a[2:].replace(".", "").replace(",", ".")) for a in quina[col]
		]
except:
	pass

In [ ]:
quina.describe()

In [ ]:
quina[[f"Bola{i}" for i in range(1, 6)]].describe()

In [ ]:
quina_fig: plt.Figure = figshow(df=quina, col="Bola",
                                name="Quina", nrows=2, ncols=3)

#### Dupla-Sena

In [ ]:
duplasena: DataFrame = resultados("Dupla-Sena")

duplasena.head()

In [ ]:
duplasena[[col for col in duplasena.columns if col.startswith("Bola")]].describe()

In [ ]:
figshow(duplasena, "Bola", "Dupla-Sena", 3, 4)

#### Dia de Sorte

In [ ]:
diasorte: DataFrame = resultados("Dia de Sorte")

diasorte.head()

#### Super Sete

In [ ]:
supersete: DataFrame = resultados("Super Sete")

supersete.head()

In [ ]:
try:
	supersete = supersete.drop(columns=["Observação"])
except:
	pass
try:
	for col in supersete.columns:
		if not col.startswith(("Rateio", "Acumulado", "Arrecadação", "Estimativa")):
			continue
		supersete[col] = [
			float(a[2:].replace(".", "").replace(",", ".")) for a in supersete[col]
		]
except:
	pass

In [ ]:
supersete.describe()

In [ ]:
supersete[[f"Coluna {i}" for i in range(1, 8)]].describe()

In [ ]:
supersete_fig: plt.Figure = figshow(df=supersete, col="Coluna ",
                                    name="Super-Sete", nrows=2, ncols=4)

#### Mais Milionária

In [ ]:
mais_milionaria: DataFrame = resultados("+Milionária")

mais_milionaria.head()

In [ ]:
try:
	mais_milionaria = mais_milionaria.drop(columns=["Observação"])
except:
	pass
try:
	for col in mais_milionaria.columns:
		if not col.startswith(("Rateio", "Acumulado", "Arrecadação", "Estimativa")):
			continue
		mais_milionaria[col] = [
			float(a[2:].replace(".", "").replace(",", ".")) for a in mais_milionaria[col]
		]
except:
	pass

In [ ]:
mais_milionaria.describe()

In [ ]:
mais_milionaria[[f"Bola{i}" for i in range(1, 7)] + ["Trevo1", "Trevo2"]].describe()

In [ ]:
mais_milionaria_fig: plt.Figure = figshow(
    df=mais_milionaria, col="Bola", name="+Milionária",
    nrows=2, ncols=4, extra="Trevo"
)

#### Resumo de Todas as Modalidades

In [ ]:
# for col in timemania.columns:
#     if not col.startswith(("Bola", "Time")): continue

#     print(col)
#     print(timemania[col].value_counts().to_dict())
#     print()
# timemania_fig

In [ ]:
for col in megasena.columns:
    if not col.startswith("Bola"): continue

    print(col)
    print(megasena[col].value_counts().to_dict())
    print()
megasena_fig

In [ ]:
for col in lotofacil.columns:
    if not col.startswith("Bola"): continue

    print(col)
    print(lotofacil[col].value_counts().to_dict())
    print()
lotofacil_fig

In [ ]:
for col in lotomania.columns:
    if not col.startswith("Bola"): continue

    print(col)
    print(lotomania[col].value_counts().to_dict())
    print()
lotomania_fig

In [ ]:
for col in quina.columns:
    if not col.startswith("Bola"): continue

    print(col)
    print(quina[col].value_counts().to_dict())
    print()
quina_fig

In [ ]:
for col in supersete.columns:
    if not col.startswith("Coluna"): continue

    print(col)
    print(supersete[col].value_counts().to_dict())
    print()
supersete_fig

In [ ]:
for col in mais_milionaria.columns:
    if not col.startswith(("Bola", "Trevo")): continue
    print(col)
    print(mais_milionaria[col].value_counts().to_dict())
    print()
mais_milionaria_fig

## Palpites

### Preparo

#### Imports

In [ ]:
from random import choices, choice

#### Constantes

In [ ]:
PRECOS: dict[str, float] = {
    "Timemania": 3.50,
    "Mega-Sena": 6.00,
    "Lotofácil": 3.50,
    "Lotomania": 3.00,
    "Quina": 3.00,
    "Dupla-Sena": 3.00,
    "Dia de Sorte": 2.50,
    "Super Sete": 3.00,
    "+Milionária": 6.00
}

#### Funções

In [ ]:
def palpite(df: DataFrame, column: str = "Bola", extra: str = "",
            minimo: int = 80) -> list[int]|list[list[int]]:
    vals: dict[str, dict[int, int]] = {}
    extras: dict[str, dict[int, int]] = {}

    for col in df.columns:
        if extra:
            if not col.startswith((column, extra)): continue
            if col.startswith(extra):
                extras[col] = df[col].value_counts().to_dict()
            else:
                vals[col] = df[col].value_counts().to_dict()
        else:
            if not col.startswith(column): continue
            vals[col] = df[col].value_counts().to_dict()
    
    for (key, val) in vals.items():
        val = {k: v for (k, v) in val.items() if v >= minimo}
        vals[key] = val
    
    if not vals:
        raise ValueError("Minimo grande demais. Diminua-o")

    for (key, val) in extras.items():
        val = {k: v for (k, v) in val.items() if v >= minimo}
        extras[key] = val

    ret: list[int]|list[list[int]] = []
    eret: list[int] = []

    while not ret or sorted(set(ret)) != ret:
        ret = []
        for val in vals.values():
            ret.append(choice(list(val.keys())))

        if any("Coluna" in c for c in vals.keys()):
            break

    if not extras:
        return ret

    while not eret or sorted(eret) != eret or any(eret.count(i) != 1 for i in eret):
        eret = []
        for val in extras.values():
                eret.append(choice(list(val.keys())))

    ret = [ret.copy(), eret]

    return ret

### Modalidades

#### Timemania

#### Mega-Sena

In [ ]:
for _ in range(5):
    print(*palpite(megasena))

#### Lotofácil

In [ ]:
for _ in range(5):
    print(palpite(lotofacil))

#### Lotomania

In [ ]:
for _ in range(5):
    print(palpite(lotomania))

#### Quina

In [ ]:
for _ in range(5):
    print(palpite(quina))

#### Dupla-Sena

#### Dia de Sorte

#### Super Sete

In [ ]:
for _ in range(5):
    print(palpite(supersete, column="Coluna"))

#### Mais Milionária

In [ ]:
for _ in range(5):
    print(palpite(mais_milionaria, extra="Trevo", minimo=10))

### Exportação

In [ ]:
from zipfile import ZipFile, ZIP_DEFLATED
from json import dumps

def export(dfs: list[tuple[str, DataFrame]], filename: str = "dumps.zip") -> None:
    data: list[BytesIO] = []

    for df in dfs:
        byte: BytesIO = BytesIO()
        values: dict = {}

        for col in df[1].columns:
            if not col.startswith(("Bola", "Coluna", "Trevo", "Time", "Mês")):
                continue
            values[col] = df[1][col].value_counts().to_dict()

        byte.write(dumps(
            values, indent=4
        ).encode())

        byte.seek(0)
        data.append(byte)

    with ZipFile(filename, mode="w", compression=ZIP_DEFLATED, compresslevel=9) as file:
        for (byte, df) in zip(data, dfs):
            file.writestr(f"{df[0]}.json", byte.read())

In [ ]:
# def palpite_json(json: ZipFile, column: str = "Bola", extra: str = "",
#             minimo: int = 80) -> list[int]|list[list[int]]:
#     vals: dict[str, dict[int, int]] = {}
#     extras: dict[str, dict[int, int]] = {}

#     for col in df.columns:
#         if extra:
#             if not col.startswith((column, extra)): continue
#             if col.startswith(extra):
#                 extras[col] = df[col].value_counts().to_dict()
#             else:
#                 vals[col] = df[col].value_counts().to_dict()
#         else:
#             if not col.startswith(column): continue
#             vals[col] = df[col].value_counts().to_dict()
    
#     for (key, val) in vals.items():
#         val = {k: v for (k, v) in val.items() if v >= minimo}
#         vals[key] = val
    
#     if not vals:
#         raise ValueError("Minimo grande demais. Diminua-o")

#     for (key, val) in extras.items():
#         val = {k: v for (k, v) in val.items() if v >= minimo}
#         extras[key] = val

#     ret: list[int]|list[list[int]] = []
#     eret: list[int] = []

#     while not ret or sorted(set(ret)) != ret:
#         ret = []
#         for val in vals.values():
#             ret.append(choice(list(val.keys())))

#         if any("Coluna" in c for c in vals.keys()):
#             break

#     if not extras:
#         return ret

#     while not eret or sorted(eret) != eret or any(eret.count(i) != 1 for i in eret):
#         eret = []
#         for val in extras.values():
#                 eret.append(choice(list(val.keys())))

#     ret = [ret.copy(), eret]

#     return ret

In [ ]:
export([("megasena", megasena), ("lotofacil", lotofacil), ("supersete", supersete)])